In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm

import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import numpy as np
import struct
from PIL import Image

In [3]:
class MNISTDataset(Dataset):
    def __init__(self, images_path, labels_path, image_size=28):
        self.images_path = images_path
        self.labels_path = labels_path
        self.image_size = image_size

        self.images = self._read_images(self.images_path)
        self.labels = self._read_labels(self.labels_path)

        self.transform = transforms.Compose([
            transforms.Resize((self.image_size, self.image_size)),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

    def _read_images(self, filepath):
        with open(filepath, 'rb') as f:
            magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
            images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, 28, 28)
        return images

    def _read_labels(self, filepath):
        with open(filepath, 'rb') as f:
            magic, num = struct.unpack(">II", f.read(8))
            labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = Image.fromarray(image, mode='L')

        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:

class MnistClassifier(nn.Module):

	def __init__(self, num_classes=10):
		super(MnistClassifier, self).__init__()

		self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
		self.pool = nn.MaxPool2d(2, 2)

		self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
		self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

		self.fc1 = nn.Linear(64 * 3 * 3, 64)
		self.fc2 = nn.Linear(64, num_classes)

	def forward(self, x):
		x = self.conv1(x)
		x = F.relu(x)
		x = self.pool(x)

		x = self.conv2(x)
		x = F.relu(x)
		x = self.pool(x)

		x = self.conv3(x)
		x = F.relu(x)
		x = self.pool(x)

		x = x.view(x.size(0), -1)
		x = self.fc1(x)
		x = F.relu(x)

		x = self.fc2(x)

		return x

In [5]:
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

train_dataset = MNISTDataset(
    images_path='/kaggle/input/mnist-dataset/train-images.idx3-ubyte',
    labels_path='/kaggle/input/mnist-dataset/train-labels.idx1-ubyte'
)

test_dataset = MNISTDataset(
    images_path='/kaggle/input/mnist-dataset/t10k-images.idx3-ubyte',
    labels_path='/kaggle/input/mnist-dataset/t10k-labels.idx1-ubyte'
)

batch_size = 16
test_batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = 10
model = MnistClassifier(num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()

learning_rate = 1e-3

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

train_losses = []
test_losses = []


Total parameters: 60874
Trainable parameters: 60874


In [6]:
epochs = 10
for epoch in range(epochs):
    model.train()
    train_loss = 0

    for data, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
        train_loss += loss.item() * data.size(0)

    model.eval()
    correct = 0
    total = 0
    test_loss = 0

    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * data.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total
    avg_train_loss = train_loss / len(train_dataset)
    avg_test_loss = test_loss / len(test_dataset)

    train_losses.append(avg_train_loss)
    test_losses.append(avg_test_loss)

    print(f"Epoch {epoch+1}: Train Loss {avg_train_loss:.4f}, Test Loss {avg_test_loss:.4f}, Test Acc {acc:.4f}")



Epoch 1: 100%|██████████| 3750/3750 [00:19<00:00, 192.73it/s]


Epoch 1: Train Loss 0.1293, Test Loss 0.0381, Test Acc 0.9871


Epoch 2: 100%|██████████| 3750/3750 [00:18<00:00, 204.56it/s]


Epoch 2: Train Loss 0.0476, Test Loss 0.0427, Test Acc 0.9857


Epoch 3: 100%|██████████| 3750/3750 [00:18<00:00, 207.28it/s]


...

Epoch 8: Train Loss 0.0156, Test Loss 0.0325, Test Acc 0.9898


Epoch 9: 100%|██████████| 3750/3750 [00:18<00:00, 206.78it/s]


Epoch 9: Train Loss 0.0120, Test Loss 0.0442, Test Acc 0.9891


Epoch 10: 100%|██████████| 3750/3750 [00:18<00:00, 206.38it/s]


Epoch 10: Train Loss 0.0140, Test Loss 0.0526, Test Acc 0.9869


In [7]:
file_path = '/kaggle/working/modelku.pth'
torch.save({
    'epoch': epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_loss': train_losses,
    'test_loss': test_losses,
}, file_path)